# początek tworzenia systemu poniżej
macie tutaj kod który utworzy wam zestaw danych do treningu

prawa autorskie do kodu (ze świetnym opisem) są Piotra Markiewicza.

# imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sn
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix ,classification_report

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Dense ,Conv2D , MaxPooling2D , Flatten , Activation , Dropout ,Lambda
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau , ModelCheckpoint, EarlyStopping

import keras
from keras import ops
from PIL import Image, ImageOps

# getting the data

In [ ]:
!git clone https://github.com/Horea94/Fruit-Images-Dataset.git

# setting variables

In [ ]:
# Parametry globalne

##############################################
learning_rate = 0.1 # initial learning rate
min_learning_rate = 0.00001 # once the learning rate reaches this value , do not decrease it further
learning_rate_reduction_factor = 0.5 # the factor used when reducing the learning rate ->
learning_rate *= learning_rate_reduction_factor
patience = 3 # how many epochs to wait before reducing the learning rate when the loss plateaus
verbose = 1 # controls the amount of logging done during training and testing: 0 - none , 1 - reports metrics after each batch , 2 - reports metrics after each epoch
image_size = (100, 100) # width and height of the used images
input_shape = (100, 100, 3) # the expected input shape for the trained models; since the images in the Fruit -360 are 100 x 100 RGB images , this is the required input shape

use_label_file = False # set this to true if you want load the label names from a file; uses the label_file defined below; the file should contain the names of the used labels , each label on a separate line
label_file = 'labels.txt'
#base_dir = '../..' # relative path to the Fruit - Images - Dataset folder
base_dir = '/content/Fruit-Images-Dataset' # relative path to the Fruit - Images - Dataset folder

test_dir = os.path.join(base_dir , 'Test')
train_dir = os.path.join(base_dir , 'Training')
output_dir = 'output_files' # root folder in which to save the the output files; the files will be under output_files/model_name
##############################################
image_size = image_size    # Wielkość wczytywanego zdjęcia = 100 x 100 pilkseli
main_folder = base_dir

# loading data

In [ ]:
# Dane do szkolen (zdjęcia warzyw i owoców) przechowywane są odpowiednio w katalogu 'Training' i 'Test', które
# mają tą samą strukturę pokazaną poniżej. Dla każdego gatunku owoca lub warzywa stworzony jest podkatalog,
# którego nazwa będzie używana jako etykieta.

# Training/Test
# |________ Etykieta 1
# |                  |____ Zdjecie 1
# |                  |____ Zdjecie 2
# |                  |____ ...
# |                  |____ Zdjecie X
# |________ Etykieta 2
# |                  |____ Zdjecie 1
# |                  |____ Zdjecie 2
# |                  |____ ...
# |                  |____ Zdjecie N
# ...
# |________ Etykieta M
# |                  |____ Zdjecie 1
# |                  |____ Zdjecie 2
# |                  |____ ...
# |                  |____ Zdjecie N


# Tworzenie ścieżek do katalogów 'Training' i 'Test'
train_image_folder = os.path.join(main_folder, 'Training')
test_image_folder = os.path.join(main_folder, 'Test')

labels = os.listdir(train_image_folder)

train_data_generator = ImageDataGenerator()
test_data_generator = ImageDataGenerator()

# Metoda Flow_from_directory klasy ImageDataGenerator umożliwia import zdjęć zapisanych w odpowiedniej strukturze
# folderów (zobacz przykład struktury przedstawionej powyżej)
'''
train_set = train_data_generator.flow_from_directory(
    train_image_folder,           # Ścieżka do katalogu ze zdjęciami
    target_size = image_size,     # Wielkość wczytywanego zdjęcia
    #class_mode = 'categorical',   # 'binary' jeżeli mamy tylko dwie etykiety, 'categorical' jeżeli jest wiele etykiet
    class_mode = 'sparse',   # 'binary' jeżeli mamy tylko dwie etykiety, 'categorical' jeżeli jest wiele etykiet
    shuffle = True,               # True - zdjęcia będą losowo mieszane w całym zbiorze
    classes = labels)             # Przypisanie etykiet do zdjęć

test_set = test_data_generator.flow_from_directory(

    test_image_folder,            # Ścieżka do katalogu ze zdjęciami
    target_size = image_size,     # Wielkość wczytywanego zdjęcia
    #class_mode = 'categorical',   # 'binary' jeżeli mamy tylko dwie etykiety, 'categorical' jeżeli jest wiele etykiet
    class_mode = 'sparse',        # 'sparse' - zgodny z modelem
    classes = labels)             # Przypisanie etykiet do zdjęć
'''

train_set = train_data_generator.flow_from_directory(
    train_image_folder,           # Ścieżka do katalogu ze zdjęciami
    target_size = image_size,     # Wielkość wczytywanego zdjęcia
    #class_mode = 'categorical',   # 'binary' jeżeli mamy tylko dwie etykiety, 'categorical' jeżeli jest wiele etykiet
    class_mode = 'sparse',   # 'binary' jeżeli mamy tylko dwie etykiety, 'categorical' jeżeli jest wiele etykiet
	  batch_size = 50,
    shuffle = True,               # True - zdjęcia będą losowo mieszane w całym zbiorze
    classes = labels)             # Przypisanie etykiet do zdjęć

test_set = test_data_generator.flow_from_directory(

    test_image_folder,            # Ścieżka do katalogu ze zdjęciami
    target_size = image_size,     # Wielkość wczytywanego zdjęcia
    #class_mode = 'categorical',   # 'binary' jeżeli mamy tylko dwie etykiety, 'categorical' jeżeli jest wiele etykiet
    class_mode = 'sparse',        # 'sparse' - zgodny z modelem
	  batch_size = 50,
	  shuffle=False,
    classes = labels)             # Przypisanie etykiet do zdjęć

In [ ]:
labels[15]

# creating/training a model

In [ ]:
keras.saving.get_custom_objects().clear()

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# if we want to train the network for a subset of the fruit classes instead of all, we can set the use_label_file to true and place in the label_file the classes we want to train for, one per line
if use_label_file:
    with open(label_file , "r") as f:
        labels = [x.strip() for x in f.readlines()]
else:
    labels = os.listdir(train_dir)
num_classes = len(labels)


# create 2 charts , one for accuracy , one for loss , to show the evolution of these two metrics during the training process
def plot_model_history(model_history , out_path=""):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    # summarize history for accuracy
    axs[0].plot(range(1, len(model_history.history['acc']) + 1), model_history.history['acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    #axs[0].set_xticks(np.arange(1, len(model_history.history['acc']) + 1), len(model_history.history['acc']))
    #axs[0].set_xticks(np.arange(1, len(model_history.history['acc']) + 1), len(model_history.history['acc']))
    axs[0].legend(['train'], loc='best')
    # summarize history for loss

    axs[1].plot(range(1, len(model_history.history['loss']) + 1), model_history.history['loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    #axs[1].set_xticks(np.arange(1, len(model_history.history['loss']) + 1), len(model_history.history['loss']))
    #axs[1].set_xticks(np.arange(1, len(model_history.history['loss']) + 1), len(model_history.history['loss']))
    axs[1].legend(['train'], loc='best')
    # save the graph in a file called "acc.png" to be available for later; the model_name is provided when creating and training a model
    if out_path:
        plt.savefig(out_path + "/acc.png")
    plt.show()


# create a confusion matrix to visually represent incorrectly classified images
def plot_confusion_matrix(y_true , y_pred , classes ,out_path=""):
    cm = confusion_matrix(y_true , y_pred)
    df_cm = pd.DataFrame(cm, index=[i for i in classes], columns=[i for i in classes])
    plt.figure(figsize=(40, 40))
    ax = sn.heatmap(df_cm , annot=True , square=True ,fmt="d", linewidths=.2, cbar_kws={"shrink":0.8})
    if out_path:
        plt.savefig(out_path + "/confusion_matrix.png") # as in the plot_model_history , the matrix is saved in a file called "model_name_confusion_matrix.png"
    return ax


# Randomly changes hue and saturation of the image to simulate variable lighting conditions

def augment_image(x):
    x = tf.image.random_saturation(x, 0.9, 1.2)
    x = tf.image.random_hue(x, 0.02)
    return x


# given the train and test folder paths and a validation to test ratio , this method creates three generators
# - the training generator uses (100 - validation_percent) of images from the train set
# it applies random horizontal and vertical flips for data augmentation and generates batches randomly
# - the validation generator uses the remaining validation_percent of images from the train set
# does not generate random batches , as the model is not trained on this data
# the accuracy and loss are monitored using the validation data so that the learning rate can be updated if the model hits a local optimum
# - the test generator uses the test set without any form of augmentation
# once the training process is done , the final values of accuracy and loss are calculated on this set
def build_data_generators(train_folder , test_folder , labels=None , image_size=(100, 100), batch_size=50):
    train_datagen = ImageDataGenerator(width_shift_range=0.0, height_shift_range=0.0, zoom_range=0.0, horizontal_flip=True ,vertical_flip=True , preprocessing_function=augment_image) # augmentation is done only on the train set (and optionally validation)

    test_datagen = ImageDataGenerator()

    train_gen = train_datagen.flow_from_directory(train_folder , target_size=image_size ,class_mode='sparse', batch_size=batch_size ,shuffle=True , subset='training', classes=labels)
    test_gen = test_datagen.flow_from_directory(test_folder , target_size=image_size , class_mode='sparse', batch_size=batch_size , shuffle=False , subset=None , classes=labels)
    return train_gen , test_gen


# Create a custom layer that converts the original image from # RGB to HSV and grayscale and concatenates the results
# forming in input of size 100 x 100 x 4
@keras.saving.register_keras_serializable(package="my_package", name="convert_to_hsv_and_grayscale")
def convert_to_hsv_and_grayscale(x):
    hsv = tf.image.rgb_to_hsv(x)
    gray = tf.image.rgb_to_grayscale(x)
    rez = tf.concat([hsv, gray], axis=-1)
    return rez

In [ ]:
# this method performs all the steps from data setup , training and testing the model and plotting the results
# the model is any trainable model; the input shape and output number of classes is dependant on the dataset used , in this case the input is 100x100 RGB images and the output is a softmax layer with 118 probabilities
# the name is used to save the classification report containing the f1 score of the model , the plots showing the loss and accuracy and the confusion matrix
# the batch size is used to determine the number of images passed through the network at once , the number of steps per epochs is derived from this as (total number of images in set // batch size) + 1
def train_and_evaluate_model(model , name="", epochs=25, batch_size=50, verbose=verbose , useCkpt=False):
    print(model.summary())
    model_out_dir = os.path.join(output_dir , name)
    if not os.path.exists(model_out_dir):
        os.makedirs(model_out_dir)
    if useCkpt:
        #model.load_weights(model_out_dir + "/model.h5")
        model.load_weights(model_out_dir + "/model.keras")

    trainGen , testGen = build_data_generators(train_dir , test_dir , labels=labels , image_size=image_size , batch_size=batch_size)
    #optimizer = Adadelta(lr=learning_rate)
    optimizer = Adadelta(learning_rate=learning_rate)
    model.compile(optimizer=optimizer , loss="sparse_categorical_crossentropy", metrics=["acc"])
    learning_rate_reduction = ReduceLROnPlateau(monitor='loss', patience=patience , verbose=verbose ,factor=learning_rate_reduction_factor,min_lr=min_learning_rate)
    #save_model = ModelCheckpoint(filepath=model_out_dir + "/model.h5", monitor='loss',verbose=verbose ,save_best_only=True ,save_weights_only=False , mode='min',save_freq='epoch')
    save_model = ModelCheckpoint(filepath=model_out_dir + "/model.keras", monitor='loss',verbose=verbose ,save_best_only=True ,save_weights_only=False , mode='min',save_freq='epoch')
    early_stopping = EarlyStopping(
        monitor='val_loss',  # Metric to monitor (e.g., validation loss)
        patience=3,          # Number of epochs with no improvement after which training will be stopped
        verbose=1,          # Verbosity mode (0: silent, 1: print messages)
        restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
    )
    history = model.fit(trainGen , epochs=epochs , steps_per_epoch=(trainGen.n // batch_size) + 1,verbose=verbose , callbacks=[learning_rate_reduction , save_model])
    #history = model.fit(trainGen , epochs=epochs , steps_per_epoch=(trainGen.n // batch_size) + 1,verbose=verbose , callbacks=[learning_rate_reduction , save_model, early_stopping])

    #model.load_weights(model_out_dir + "/model.h5")
    model.load_weights(model_out_dir + "/model.keras")
    '''
    trainGen.reset()
    loss_t , accuracy_t = model.evaluate(trainGen ,steps=(trainGen.n // batch_size) + 1, verbose=verbose)
    loss , accuracy = model.evaluate(testGen , steps=(testGen.n // batch_size) + 1, verbose=verbose)
    print("Train: accuracy = %f ; loss_v = %f" % (accuracy_t , loss_t))
    print("Test: accuracy = %f ; loss_v = %f" % (accuracy , loss))
    plot_model_history(history , out_path=model_out_dir)
    testGen.reset()
    '''
    y_pred = model.predict(testGen , steps=(testGen.n// batch_size) + 1, verbose=verbose)
    '''
    y_true = testGen.classes[testGen.index_array]
    plot_confusion_matrix(y_true , y_pred.argmax(axis=-1), labels , out_path=model_out_dir)
    class_report = classification_report(y_true ,y_pred.argmax(axis=-1), target_names=labels)

    with open(model_out_dir + "/classification_report.txt", "w") as text_file:
        text_file.write("%s" % class_report)
    '''

In [ ]:
#model = tf.keras.models.Sequential()

# tutaj musicie wstawić własny kod
# .......
model = tf.keras.Sequential([
Input(shape=input_shape , name='data'),
Lambda(convert_to_hsv_and_grayscale, output_shape=(100, 100, 4)),
Conv2D(16, (5, 5), strides=(1, 1), padding='same', name='conv1'),
Activation('relu', name='conv1_relu'),
MaxPooling2D((2, 2), strides=(2, 2), padding='valid', name='pool1'),
Conv2D(32, (5, 5), strides=(1, 1), padding='same', name='conv2'),
Activation('relu', name='conv2_relu'),
MaxPooling2D((2, 2), strides=(2, 2), padding='valid', name='pool2'),
Conv2D(64, (5, 5), strides=(1, 1), padding='same', name='conv3'),
Activation('relu', name='conv3_relu'),
MaxPooling2D((2, 2), strides=(2, 2), padding='valid', name='pool3'),
Conv2D(128, (5, 5), strides=(1, 1), padding='same', name='conv4'),
Activation('relu', name='conv4_relu'),
MaxPooling2D((2, 2), strides=(2, 2), padding='valid', name='pool4'),
Flatten(),
Dense(1024, activation='relu', name='fcl1'),
Dropout(0.2),
Dense(256, activation='relu', name='fcl2'),
Dropout(0.2),
Dense(num_classes , activation='softmax', name='predictions')
])

#model.compile(# tutaj musicie wstawić własny kod)

In [ ]:
#model.fit(# tutaj musicie wstawić własny kod)
batch_size=50
model_name = 'fruit-360_model'
model_out_dir = os.path.join(output_dir , model_name)
epochs=25
#useCkpt=False
useCkpt=True
if not os.path.exists(model_out_dir):
    os.makedirs(model_out_dir)
if useCkpt:
    #model.load_weights(model_out_dir + "/model.h5")
    model.load_weights(model_out_dir + "/model.keras")
trainGen , testGen = build_data_generators(train_dir , test_dir , labels=labels , image_size=image_size , batch_size=batch_size)
#optimizer = Adadelta(lr=learning_rate)
optimizer = Adadelta(learning_rate=learning_rate)
model.compile(optimizer=optimizer , loss="sparse_categorical_crossentropy", metrics=["acc"])
model.summary()
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', patience=patience , verbose=verbose ,factor=learning_rate_reduction_factor,min_lr=min_learning_rate)
#save_model = ModelCheckpoint(filepath=model_out_dir + "/model.h5", monitor='loss',verbose=verbose ,save_best_only=True ,save_weights_only=False , mode='min',save_freq='epoch')
save_model = ModelCheckpoint(filepath=model_out_dir + "/model.keras", monitor='loss',verbose=verbose ,save_best_only=True ,save_weights_only=False , mode='min',save_freq='epoch')
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,          # Verbosity mode (0: silent, 1: print messages)
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)
#history = model.fit(trainGen , epochs=epochs , steps_per_epoch=(trainGen.n // batch_size) + 1,verbose=verbose , callbacks=[learning_rate_reduction , save_model])
history = model.fit(train_set , epochs=epochs , steps_per_epoch=(train_set.n // batch_size) + 1,verbose=verbose , callbacks=[learning_rate_reduction , save_model])
model.load_weights(model_out_dir + "/model.keras")
y_pred = model.predict(testGen , steps=(testGen.n// batch_size) + 1, verbose=verbose)

In [ ]:
model.summary()

# model evaluation

In [ ]:
#model.evaluate(# tutaj musicie wstawić własny kod)
loss, accuracy = model.evaluate(test_set)
print("Loss:", loss)
print("Accuracy:", accuracy)


In [ ]:
'''
print(labels)
print(num_classes)
model = network(input_shape=input_shape , num_classes=num_classes)
#train_and_evaluate_model(model , name="fruit-360_model")
#train_and_evaluate_model(model , name="fruit-360_model", useCkpt=True)
train_and_evaluate_model(model , name="fruit-360_model", epochs=10)
'''

## model prediction

In [ ]:
!wget -O banana.jpg https://upload.wikimedia.org/wikipedia/commons/8/8a/Banana-Single.jpg

In [ ]:
# preprocessing data (przygotowujemy zdjęcie z internetu do predykcji

image = Image.open("banana.jpg").convert("RGB")
image = image.resize((100, 100))

image_np = np.array(image)

img = np.array([image])
#img = image_np.reshape(1, 100, 100, 3).astype('float32') / 255.0

#plt.imshow(image/255)
plt.imshow(image_np/255)
print(img.shape)

In [ ]:
#predykcja = # wasz kod tutaj
predykcja = model.predict(img)

predykcja
print(predykcja)

In [ ]:
# kod poniżej wyświetli nam nazwę rozpoznanej kategorii owocu
print(np.argmax(predykcja))
print(labels)
#labels = os.listdir(train_image_folder)
#labels[np.argmax(predykcja)]
labels[np.argmax(predykcja)]
print(labels[np.argmax(predykcja)])

In [ ]:
img4 = tf.keras.preprocessing.image.load_img('banana.jpg', target_size=(100, 100))
img4 = tf.keras.preprocessing.image.img_to_array(img4)
plt.imshow(img4.astype(np.int32))
print(img4.shape)

image4 = np.array([img4])

labels[np.argmax(model.predict(image4))]
print(labels[np.argmax(model.predict(image4))])

# predykcja używając zdjęcia z danych testowych

In [ ]:
# prediction using picture from the dataset

img2 = tf.keras.preprocessing.image.load_img('/content/Fruit-Images-Dataset/Test/Apple Red Delicious/0_100.jpg', target_size=(100, 100))
img2 = tf.keras.preprocessing.image.img_to_array(img2)
plt.imshow(img2.astype(np.int32))
print(img2.shape)

image2 = np.array([img2])

labels[np.argmax(model.predict(image2))]

In [ ]:
model.save("/content/output_files/fruit-360_model/model.keras")

In [ ]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("/content/output_files/fruit-360_model/model.keras")

# Let's check:
np.testing.assert_allclose(
    model.predict(image4), reconstructed_model.predict(image4)
)
print(labels[np.argmax(model.predict(image4))])
print(labels[np.argmax(reconstructed_model.predict(image4))])

In [ ]:


model.save_weights("/content/output_files/fruit-360_model/model.weights.h5")

In [ ]:

# prompt: generate a csv file with labels from the labels variable

import csv

# Assuming 'labels' variable is defined as in the provided code
# labels = os.listdir(train_image_folder)

with open('labels.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['label'])  # Write header
    for label in labels:
        writer.writerow([label])
